In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


In [ ]:

# Step 0: Imports & Setup

import torch, random, json, time
from pathlib import Path
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler
from PIL import Image

# Folders
PROJECT = "landmark_cuda_run"
OUT_DIR = Path(PROJECT) / "outputs"
META_DIR = Path(PROJECT) / "meta"
for p in [OUT_DIR, META_DIR]: p.mkdir(parents=True, exist_ok=True)

# Force GPU
assert torch.cuda.is_available(), "CUDA GPU not available. Install NVIDIA drivers + CUDA-enabled PyTorch."
device = "cuda"
print("Running on:", torch.cuda.get_device_name(0))

# Seed
def set_seed(seed: int = 1234):
    import numpy as np
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)


In [ ]:

# Step 1: Load SDXL on GPU

MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"


pipe = StableDiffusionXLPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,   
    variant="fp16",
    use_safetensors=True
).to(device)

pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

# Memory optimizations

pipe.enable_vae_slicing()
pipe.enable_vae_tiling()

print("✅ Pipeline loaded on GPU")


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  2.71it/s]


✅ Pipeline loaded on GPU


In [ ]:

# Step 2: Define Prompts

locations = {
    "Mumbai Bandra-Worli Sea Link": [
        "front view from sea level, full span visible, golden-hour light",
        "side view from the bridge approach, monsoon clouds, wet asphalt reflections",
        "aerial drone shot at night, bridge lights forming arc, Mumbai skyline"
    ],
    "Rajasthan Hawa Mahal": [
        "frontal facade in daylight, pink sandstone, honeycomb windows, street foreground",
        "side street perspective, bustling bazaar, tourists walking",
        "aerial view of Jaipur with Hawa Mahal centered, Aravalli hills in distance"
    ],
    "Nagpur rainforest": [
        "ground-level trail, dense monsoon canopy, beams of sunlight, mist",
        "side angle near waterfall, wet rocks, moss, colorful birds",
        "aerial canopy after rain, rolling green treetops, low clouds"
    ],
}

STYLE = "Photorealistic, ultra-detailed, natural colors, sharp focus, 8k"
NEG = "blurry, distorted, cartoonish, lowres, text, watermarks"


In [ ]:

# Step 3: Generate All Images

HEIGHT, WIDTH = 1024, 1024   
NUM_STEPS = 30
GUIDANCE = 7

records = []

for place, angle_list in locations.items():
    for angle_text in angle_list:
        seed = random.randint(1, 10_000_000)
        set_seed(seed)

        prompt = f"{STYLE}. {place}, {angle_text}"
        file_stub = f"{place.replace(' ', '_')}_{angle_text[:20].replace(' ','_')}_{seed}"

        print(f"\n🎨 Generating → {place} | {angle_text} | Seed: {seed}")
        start = time.time()

        image = pipe(
            prompt=prompt,
            negative_prompt=NEG,
            num_inference_steps=NUM_STEPS,
            guidance_scale=GUIDANCE,
            height=HEIGHT, width=WIDTH
        ).images[0]

        end = time.time()
        print(f"✅ Done in {(end-start):.2f} sec")

        out_path = OUT_DIR / f"{file_stub}.png"
        image.save(out_path)

        meta = {
            "place": place, "angle": angle_text, "prompt": prompt,
            "negative_prompt": NEG, "seed": seed, "steps": NUM_STEPS,
            "guidance": GUIDANCE, "size": [WIDTH, HEIGHT],
            "timestamp": int(time.time())
        }
        with open(META_DIR / f"{file_stub}.json", "w") as f:
            json.dump(meta, f, indent=2)

        records.append({"image": str(out_path), **meta})

print("\n🎉 Total images generated:", len(records))



🎨 Generating → Mumbai Bandra-Worli Sea Link | front view from sea level, full span visible, golden-hour light | Seed: 2818325


100%|██████████| 30/30 [04:03<00:00,  8.11s/it]


✅ Done in 266.16 sec

🎨 Generating → Mumbai Bandra-Worli Sea Link | side view from the bridge approach, monsoon clouds, wet asphalt reflections | Seed: 5637584


100%|██████████| 30/30 [08:22<00:00, 16.76s/it]


✅ Done in 511.17 sec

🎨 Generating → Mumbai Bandra-Worli Sea Link | aerial drone shot at night, bridge lights forming arc, Mumbai skyline | Seed: 835762


...

✅ Done in 512.11 sec

🎨 Generating → Nagpur rainforest | side angle near waterfall, wet rocks, moss, colorful birds | Seed: 7794082


100%|██████████| 30/30 [08:23<00:00, 16.77s/it]


✅ Done in 511.72 sec

🎨 Generating → Nagpur rainforest | aerial canopy after rain, rolling green treetops, low clouds | Seed: 2975810


100%|██████████| 30/30 [08:11<00:00, 16.40s/it]


✅ Done in 500.08 sec

🎉 Total images generated: 9


In [ ]:

from realesrgan import RealESRGAN

# Initialize ESRGAN (x4 upscale model)
device_re = torch.device("cuda" if torch.cuda.is_available() else "cpu")
upscaler = RealESRGAN(device_re, scale=4)

# Download pretrained weights (first time only)
upscaler.load_weights("weights/RealESRGAN_x4plus.pth")

UPSCALED_DIR = Path(PROJECT) / "outputs_upscaled"
UPSCALED_DIR.mkdir(parents=True, exist_ok=True)

print("\n🚀 Upscaling images...")

for img_file in OUT_DIR.glob("*.png"):
    img = Image.open(img_file).convert("RGB")
    sr = upscaler.predict(img)   # 4× upscaling
    sr_resized = sr.resize((2048, 2048))  # resize to clean 2K
    sr_resized.save(UPSCALED_DIR / img_file.name)

print("✅ Upscaling complete. Files saved in:", UPSCALED_DIR)


✅ Upscaling complete. Files saved in: landmark_cuda_run\outputs_upscaled


In [ ]:

from realesrgan import RealESRGAN

# Initialize ESRGAN (x4 upscale model)
device_re = torch.device("cuda" if torch.cuda.is_available() else "cpu")
upscaler = RealESRGAN(device_re, scale=4)

# Download pretrained weights (first time only)
upscaler.load_weights("weights/RealESRGAN_x4plus.pth")

UPSCALED_DIR = Path(PROJECT) / "outputs_upscaled"
UPSCALED_DIR.mkdir(parents=True, exist_ok=True)

print("\n🚀 Upscaling images...")

for img_file in OUT_DIR.glob("*.png"):
    img = Image.open(img_file).convert("RGB")
    sr = upscaler.predict(img)   # 4× upscaling
    sr_resized = sr.resize((2048, 2048))  # resize to clean 2K
    sr_resized.save(UPSCALED_DIR / img_file.name)

print("✅ Upscaling complete. Files saved in:", UPSCALED_DIR)
